In [23]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [24]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))


def plot_grid(
    grid: list[list[str]],
    colors: dict[str, int],
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((len(grid), len(grid[0])))
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char in colors:
                arr[r, c] = colors[char]
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)


def plot_objects(
    object_lists: list[list[tuple[int, int]]],
    colors: list[int],
    x_limit: int,
    y_limit: int,
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((y_limit, x_limit))
    for objects, color in zip(object_lists, colors):
        for obj in objects:
            arr[y_limit - 1 - obj[1], obj[0]] = color
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)

In [25]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex"
today

'day20'

In [26]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines


def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid


def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]


def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]


def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [27]:
grid = get_grid()

start = None
end = None
for r in range(len(grid)):
    for c in range(len(grid[r])):
        if grid[r][c] == "S":
            start = (r, c)
        elif grid[r][c] == "E":
            end = (r, c)

len(grid), len(grid[0]), start, end

(141, 141, (113, 87), (89, 83))

In [29]:
assert start is not None
assert end is not None

distances_from_start = {}
dq = deque([(start, 0)])
while dq:
    pos, d = dq.pop()
    
    if pos in distances_from_start:
        continue
    distances_from_start[pos] = d

    for dr, dc in dirs4:
        if grid[pos[0] + dr][pos[1] + dc] == "#":
            continue
        next_pos = (pos[0] + dr, pos[1] + dc)
        dq.appendleft((next_pos, d + 1))

distances_from_end = {}
dq = deque([(end, 0)])
while dq:
    pos, d = dq.pop()
    
    if pos in distances_from_end:
        continue
    distances_from_end[pos] = d

    for dr, dc in dirs4:
        if grid[pos[0] + dr][pos[1] + dc] == "#":
            continue
        next_pos = (pos[0] + dr, pos[1] + dc)
        dq.appendleft((next_pos, d + 1))

shortest = distances_from_start[end]
shortest

9408

In [ ]:
count = 0
for r in range(1, len(grid) - 1):
    for c in range(1, len(grid[r]) - 1):
        if grid[r][c] == "#":
            continue
        dist_to_cheat = distances_from_start[(r, c)]
        for dr, dc in dirs4:
            cheat_end = (r+dr, c+dc)
            dist_from_cheat_end = 10000000000
            for dr2, dc2 in dirs4:
                if (r+dr+dr2, c+dc+dc2) not in distances_from_end:
                    continue
                dist_from_cheat_end = min(dist_from_cheat_end, distances_from_end[(r+dr+dr2, c+dc+dc2)])
            if dist_to_cheat + dist_from_cheat_end + 1 < shortest - 100:
                count += 1
count

1365

In [37]:
count = 0
for r in range(1, len(grid) - 1):
    for c in range(1, len(grid[r]) - 1):
        for r2 in range(max(1, r - 20), min(len(grid) - 1, r + 21)):
            for c2 in range(max(1, c - 20), min(len(grid[r]) - 1, c + 21)):
                if abs(r-r2) + abs(c-c2) > 20:
                    continue
                cheat_start = (r, c)
                cheat_end = (r2, c2)
                if cheat_start not in distances_from_start:
                    continue
                if cheat_end not in distances_from_end:
                    continue
                if distances_from_start[cheat_start] + distances_from_end[cheat_end] + abs(r-r2) + abs(c-c2) < shortest - 99:
                    count += 1
count

986082